In [6]:
from openai import OpenAI
from dotenv import  load_dotenv , find_dotenv
import os

_ : bool = load_dotenv(find_dotenv())

client : OpenAI = OpenAI()


In [7]:
response = client.chat.completions.create(
    messages = [{
              "role":"user" ,
              "content" : "who are you?"
              }],
    model = "gpt-3.5-turbo-1106"
)

print(response)
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-8RWbcdrH36Ozlpg8anZQ8rQTqY2QU', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='I am a language model AI created by OpenAI, designed to assist and provide information on various topics. How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1701570884, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_eeff13170a', usage=CompletionUsage(completion_tokens=28, prompt_tokens=11, total_tokens=39))
I am a language model AI created by OpenAI, designed to assist and provide information on various topics. How can I assist you today?


In [8]:
from openai.types.chat.chat_completion import ChatCompletion

def chat_completion(prompt : str )-> str:
 response : ChatCompletion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-3.5-turbo-1106",
    )
# print(response)
#  print(response.choices[0].message.content)
 return response.choices[0].message.content

chat_completion("what is 1+1?")

'1+1 equals 2.'

## Multiple Roles

In [9]:
def chat_completion()->str:
  completion : ChatCompletion = client.chat.completions.create(
    model  = "gpt-3.5-turbo-1106",
    messages= [
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )

  return completion.choices[0].message.content

print(chat_completion())

In the land of code, where algorithms reign,
There lies a concept both simple and arcane,
Recursion it's called, a mystical spell,
That dances through functions, a tale to tell.

Like a mirror reflecting its own reflection,
Recursion calls itself, without objection,
A function that calls itself, again and again,
A loop in disguise, it's quite the brain gain.

For when a problem breaks into smaller bits,
Recursion steps in, and courage it admits,
It dives deep within, solving one by one,
Till the puzzle is solved, and the victory won.

Like a Russian doll nested in layers of lore,
Recursion unwinds, unraveling much more,
Till the base case is found, the end of the line,
And the code unwinds gracefully, oh so divine.

So embrace recursion, with its mesmerizing charm,
For in the world of programming, it's more than just a farm,
It's a powerful tool, a clever coding friend,
That brings elegance and grace to code, from beginning to end.


## streaming

In [11]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True, # mark the stream true to turn on streaming
)


for part in stream:
    print(part.choices[0].delta.content or "")



This
 is
 a
 test
.



### getting response in json format

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={ "type": "json_object" }, # Adding response Format as json_object
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},#also don't forget to mention json here
    {"role": "user", "content": "List of months that have 30 days"}
  ]
)
print(response.choices[0].message.content)

{
  "months": ["April", "June", "September", "November"]
}


#### parsing the structured string into json

In [ ]:
import json

# parse response:
obj: dict[str, list[str]] = json.loads(response.choices[0].message.content)

# the result is a Python dictionary:
print(obj)
print(obj['months'])

{'months': ['April', 'June', 'September', 'November']}
['April', 'June', 'September', 'November']


## Function Calling


In [12]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location:str, unit:str="fahrenheit")->str:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})



In [16]:
import json
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        
        messages.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        display("* Second Request Messages: ", list(messages))
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content

In [17]:
run_conversation("What's the weather like in San Francisco, Tokyo, and Paris?")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_yg8k9Pok0xXI3pV8LLgGV4eI', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_NT9ThKfaS8BesOSSWYuEDvwE', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_dTijzjRUMJryyWcZMalJZxA5', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_yg8k9Pok0xXI3pV8LLgGV4eI', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_NT9ThKfaS8BesOSSWYuEDvwE', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_dTijzjRUMJryyWcZMalJZxA5', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]

'* Second Request Messages: '

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yg8k9Pok0xXI3pV8LLgGV4eI', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_NT9ThKfaS8BesOSSWYuEDvwE', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_dTijzjRUMJryyWcZMalJZxA5', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_yg8k9Pok0xXI3pV8LLgGV4eI',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'},
 {'tool_call_id': 'call_NT9ThKfaS8BesOSSWYuEDvwE',
  'role': 

* Second Response:  {'id': 'chatcmpl-8RWl9n3dff21OK10RPjIaaIS1PjMj', 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Currently, in San Francisco, the weather is 72°F and partly cloudy. In Tokyo, the weather is 10°C and rainy. In Paris, the weather is 22°C and mostly sunny.', role='assistant', function_call=None, tool_calls=None))], 'created': 1701571475, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_eeff13170a', 'usage': CompletionUsage(completion_tokens=41, prompt_tokens=169, total_tokens=210)}


'Currently, in San Francisco, the weather is 72°F and partly cloudy. In Tokyo, the weather is 10°C and rainy. In Paris, the weather is 22°C and mostly sunny.'